In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-clean-unbalanced/Dataset_clean_unbalanced.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2012_to_2014.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2009_to_2011.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2005_to_2007.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accident_coords_update.ipynb
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/ukTrafficAADF.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/Areas.shp
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/Local_Authority_Districts_Dec_2016.geojson


In [ ]:
dataset1=pd.read_csv('/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2005_to_2007.csv')
dataset2=pd.read_csv('/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2009_to_2011.csv')
dataset3=pd.read_csv('/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2012_to_2014.csv')

In [ ]:
dataset1 = pd.concat([dataset1,dataset2,dataset3],ignore_index=True)
dataset1.info()
#dataset2.info()
#dataset3.info()

In [ ]:
df1=dataset1.drop(['Accident_Index',
                   'Local_Authority_(District)',
                   'Local_Authority_(Highway)',
                   '1st_Road_Number',
                   'Junction_Detail',
                   '2nd_Road_Number',
                   'Year',
                   'Location_Easting_OSGR',
                   'Location_Northing_OSGR',
                   'Longitude',
                   'Latitude',
                   'LSOA_of_Accident_Location'
                  ],axis=1)

In [ ]:
df1.info()

In [ ]:
# en faisant la soustraction des 'True' de verif2 et verif3, on obtient le nbr de mesure où 
# il y a 2nd_Road_Class != -1 (donc il y a une route secondaire) mais 
# il n'y a pas de descriptif du control de l'intersection.
#
# De plus, le résultat de cette soustraction est positif donc
# toutes les lignes avec 2nd_Road_Class == -1 correspondent bien à Junction_Control == Null

In [ ]:
#True si il y a une 2nd_Road mais pas de description du control de l'intersection
#verif4=df1.apply(lambda x:True if x['2nd_Road_Class'] != -1 and x['Junction_Control']!='Authorised person' and x['Junction_Control']!='Giveway or uncontrolled' and x['Junction_Control']!='Automatic traffic signal' and x['Junction_Control']!='Stop Sign' else False, axis=1)
#verif4.value_counts()

In [ ]:
df1['verif']=df1.apply(lambda x:True if x['2nd_Road_Class'] != -1 and x['Junction_Control']!='Authorised person' and x['Junction_Control']!='Giveway or uncontrolled' and x['Junction_Control']!='Automatic traffic signal' and x['Junction_Control']!='Stop Sign' else False, axis=1)
df1.info()

In [ ]:
df1['Junction_Control'].value_counts()

In [ ]:
df1.drop(df1[df1['verif']==True].index,inplace=True)
df1.info()

In [ ]:
df1['Junction_Control'].value_counts()

In [ ]:
df1['Junction_Control']=df1['Junction_Control'].fillna('No_Junction')

In [ ]:
df1['Junction_Control'].value_counts()

In [ ]:
df1.drop(['verif'],axis=1,inplace=True)
df1.info()

In [ ]:
df1['Road_Type'].value_counts()

In [ ]:
df1['Light_Conditions'].value_counts()

In [ ]:
df1['Weather_Conditions'].value_counts()

In [ ]:
df1['verif_NaN']=df1.apply(lambda x:True if pd.isna(x['Time']) or pd.isna(x['Pedestrian_Crossing-Human_Control']) or pd.isna(x['Pedestrian_Crossing-Physical_Facilities']) or pd.isna(x['Weather_Conditions']) or pd.isna(x['Road_Surface_Conditions']) or pd.isna(x['Special_Conditions_at_Site']) or pd.isna(x['Carriageway_Hazards']) or pd.isna(x['Did_Police_Officer_Attend_Scene_of_Accident']) else False, axis=1)
df1.info()

In [ ]:
df1['verif_NaN'].value_counts()

In [ ]:
df1.drop(df1[df1['verif_NaN']==True].index,inplace=True)
df1.info()

In [ ]:
df1.drop(['verif_NaN'],axis=1,inplace=True)
df1.info()

In [ ]:
df1['verif_Unknown']=df1.apply(lambda x:True if x['Road_Type']=='Unknown' or x['Light_Conditions']=='Darkness: Street lighting unknown' or x['Weather_Conditions']=='Unknown' else False, axis=1)
df1['verif_Unknown'].value_counts()

In [ ]:
df1.drop(df1[df1['verif_Unknown']==True].index,inplace=True)
df1.drop(['verif_Unknown'],axis=1,inplace=True)
df1.info()

In [ ]:
#df1.to_csv('Dataset_clean.csv')

In [ ]:
df1['Date'].head()

In [ ]:
eclat = df1['Date'].str.split("/",n=2,expand=True)
eclat.head()

In [ ]:
eclat['mois']=eclat.apply(lambda x:float(x[1]) if x[0]<'15' else float(x[1])+0.5, axis=1)

In [ ]:
eclat.head(10)

In [ ]:
df1['Mois']=eclat['mois']
df1.drop(columns =['Date'], inplace = True)
df1.info()

In [ ]:
eclat = df1['Time'].str.split(":",n=1,expand=True)
eclat.head(10)

In [ ]:
eclat['decimal']=eclat.apply(lambda x:float(x[0])+(float(x[1])/60), axis=1)
eclat.head(10)

In [ ]:
df1['Heure']=eclat['decimal']
df1.drop(columns =['Time'], inplace = True)
df1.info()

In [ ]:
df1['Did_Police_Officer_Attend_Scene_of_Accident'].unique()

In [ ]:
#faire mappage des colonnes de type object
df1['Road_Type']=df1['Road_Type'].map({'Single carriageway':0,
                                       'Dual carriageway':1,
                                       'One way street':2,
                                       'Roundabout':3,
                                       'Slip road':4})
df1['Junction_Control']=df1['Junction_Control'].map({'No_Junction':0,
                                                     'Automatic traffic signal':1,
                                                     'Giveway or uncontrolled':2,
                                                     'Stop Sign':3,
                                                     'Authorised person':4})
df1['Pedestrian_Crossing-Human_Control']=df1['Pedestrian_Crossing-Human_Control'].map({'None within 50 metres':0,
                                                                                       'Control by other authorised person':1,
                                                                                       'Control by school crossing patrol':2})
df1['Pedestrian_Crossing-Physical_Facilities']=df1['Pedestrian_Crossing-Physical_Facilities'].map({'Zebra crossing':0,
                                                                                                   'Pedestrian phase at traffic signal junction':1,
                                                                                                   'No physical crossing within 50 meters':2,
                                                                                                   'Central refuge':3,
                                                                                                   'non-junction pedestrian crossing':4,
                                                                                                   'Footbridge or subway':5})
df1['Light_Conditions']=df1['Light_Conditions'].map({'Daylight: Street light present':0,
                                                     'Darkness: Street lights present and lit':1,
                                                     'Darkness: Street lights present but unlit':2,
                                                     'Darkeness: No street lighting':3})
df1['Weather_Conditions']=df1['Weather_Conditions'].map({'Raining without high winds':0,
                                                         'Fine without high winds':1,
                                                         'Snowing without high winds':2,
                                                         'Other':3,
                                                         'Fine with high winds':4,
                                                         'Raining with high winds':5,
                                                         'Fog or mist':6,
                                                         'Snowing with high winds':7})
df1['Road_Surface_Conditions']=df1['Road_Surface_Conditions'].map({'Wet/Damp':0,
                                                                   'Dry':1,
                                                                   'Frost/Ice':2,
                                                                   'Snow':3,
                                                                   'Flood (Over 3cm of water)':4})
df1['Special_Conditions_at_Site']=df1['Special_Conditions_at_Site'].map({'None':0,
                                                                         'Ol or diesel':1,
                                                                         'Roadworks':2,
                                                                         'Auto traffic signal partly defective':3,
                                                                         'Road surface defective':4,
                                                                         'Auto traffic singal out':5,
                                                                         'Permanent sign or marking defective or obscured':6,
                                                                         'Mud':7})
df1['Carriageway_Hazards']=df1['Carriageway_Hazards'].map({'None':0,
                                                           'Other object in carriageway':1,
                                                           'Pedestrian in carriageway (not injured)':2,
                                                           'Dislodged vehicle load in carriageway':3,
                                                           'Involvement with previous accident':4,
                                                           'Any animal (except a ridden horse)':5})
df1['Did_Police_Officer_Attend_Scene_of_Accident']=df1['Did_Police_Officer_Attend_Scene_of_Accident'].map({'Yes':0,
                                                                                                           'No':1})
df1.info()

In [ ]:
df1['Severity']=df1['Accident_Severity']
df1.drop(columns =['Accident_Severity'], inplace = True)
df1.info()

In [ ]:
df1.to_csv('Dataset_clean_unbalanced.csv')

In [ ]:
#Objectif : mis en place d'un model prédictif sur la sévérité d'un accident

# donnée de sortie (Y) : Accident_Severity

# données sans intérêt : Accident_Index , Local_Authority_(District) , Local_Authority_(Highway) , 1st_Road_Number , Junction_Detail ,
#                        2nd_Road_Number , Year , Location_Easting_OSGR , Location_Northing_OSGR , Longitude , Latitude , 
#                        LSOA_of_Accident_Location

# données d'intérêt (X) : Police_Force , Number_of_Vehicles , Number_of_Casualties , Date , Day_of_Week , Time ,
#                         1st_Road_Class , Road_Type , Speed_limit , Pedestrian_Crossing-Human_Control , Pedestrian_Crossing-Physical_Facilities ,
#                         Light_Conditions , Weather_Conditions , Road_Surface_Conditions , Special_Conditions_at_Site , Carriageway_Hazards ,
#                         Urban_or_Rural_Area , Did_Police_Officer_Attend_Scene_of_Accident , Junction_Control

'''
TODO :  dataset déséquilivré, faire 1ère analyse pour comparer avant/après ré-équilibrage
'''

In [9]:
df1=pd.read_csv('/kaggle/input/dataset-clean-unbalanced/Dataset_clean_unbalanced.csv')
df1.drop(columns =['Unnamed: 0'], inplace = True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452458 entries, 0 to 1452457
Data columns (total 21 columns):
 #   Column                                       Non-Null Count    Dtype  
---  ------                                       --------------    -----  
 0   Police_Force                                 1452458 non-null  int64  
 1   Number_of_Vehicles                           1452458 non-null  int64  
 2   Number_of_Casualties                         1452458 non-null  int64  
 3   Day_of_Week                                  1452458 non-null  int64  
 4   1st_Road_Class                               1452458 non-null  int64  
 5   Road_Type                                    1452458 non-null  int64  
 6   Speed_limit                                  1452458 non-null  int64  
 7   Junction_Control                             1452458 non-null  int64  
 8   2nd_Road_Class                               1452458 non-null  int64  
 9   Pedestrian_Crossing-Human_Control            1

In [13]:
import matplotlib as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [10]:
X = df1.loc[:, df1.columns!='Severity']
Y = df1['Severity']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [12]:
df1.corr()

,Police_Force,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,1st_Road_Class,Road_Type,Speed_limit,Junction_Control,2nd_Road_Class,Pedestrian_Crossing-Human_Control,...,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,Mois,Heure,Severity
Police_Force,1.000000,-0.014733,0.008196,0.000433,0.043108,0.022900,0.200911,-0.104980,-0.119040,0.032742,...,0.058672,0.034852,-0.037199,0.024343,0.031176,0.242774,-0.013277,0.001658,-0.013189,-0.032511
Number_of_Vehicles,-0.014733,1.000000,0.236753,-0.000030,-0.140527,0.054965,0.083436,0.075180,0.067437,-0.018949,...,-0.090726,-0.015269,-0.012844,-0.012012,-0.028659,0.040603,-0.030405,0.006557,0.013064,0.075870
Number_of_Casualties,0.008196,0.236753,1.000000,-0.001352,-0.082282,-0.003548,0.139471,-0.032653,-0.030295,-0.008204,...,0.050067,0.001934,-0.029941,0.012502,0.008493,0.116414,-0.103628,0.001645,0.026175,-0.083500
Day_of_Week,0.000433,-0.000030,-0.001352,1.000000,0.007061,-0.006527,-0.015985,0.005670,0.007955,-0.001170,...,0.004877,-0.002584,-0.004430,-0.002411,-0.003658,-0.016594,0.004030,0.003730,0.028199,0.003042
1st_Road_Class,0.043108,-0.140527,-0.082282,0.007061,1.000000,-0.244051,-0.388813,0.086551,0.106371,0.015590,...,-0.053110,-0.005761,0.018384,0.006264,-0.015128,-0.226674,0.084066,-0.004160,0.036358,0.012599
Road_Type,0.022900,0.054965,-0.003548,-0.006527,-0.244051,1.000000,0.158762,0.125561,0.042656,-0.007922,...,-0.024530,-0.001965,-0.007617,-0.008318,-0.013396,0.045029,0.031935,0.002367,-0.018703,0.049781
Speed_limit,0.200911,0.083436,0.139471,-0.015985,-0.388813,0.158762,1.000000,-0.275316,-0.338097,-0.039551,...,0.197736,0.055506,-0.037927,0.055080,0.078850,0.687139,-0.155257,0.002468,-0.047365,-0.078973
Junction_Control,-0.104980,0.075180,-0.032653,0.005670,0.086551,0.125561,-0.275316,1.000000,0.906350,0.015091,...,-0.127318,-0.028621,-0.001717,-0.038766,-0.071763,-0.226985,0.042561,-0.001230,0.020175,0.067452
2nd_Road_Class,-0.119040,0.067437,-0.030295,0.007955,0.106371,0.042656,-0.338097,0.906350,1.000000,0.018250,...,-0.126737,-0.032067,-0.002275,-0.034784,-0.072571,-0.296841,0.037095,-0.001608,0.027280,0.064871
Pedestrian_Crossing-Human_Control,0.032742,-0.018949,-0.008204,-0.001170,0.015590,-0.007922,-0.039551,0.015091,0.018250,1.000000,...,-0.019952,-0.000247,-0.000286,-0.002239,-0.001909,-0.032171,0.013694,-0.001842,-0.010040,0.007604


In [14]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier()
classifierKNN.fit(X_train, y_train)
y_predKNN = classifierKNN.predict(X_test)
#accuracy
accKNN = accuracy_score(y_test, y_predKNN)
print("accuracy_score =",accKNN)
#confusion matrix
cmKNN = confusion_matrix(y_test, y_predKNN)
matrice_confKNN=pd.DataFrame(cmKNN,index=['1','2','3'],columns=['1','2','3'])
matrice_confKNN
#report
print(classification_report(y_test,y_predKNN))

accuracy_score = 0.8295443592250389


ValueError: Shape of passed values is (3, 3), indices imply (2, 2)

In [16]:
#confusion matrix
cmKNN = confusion_matrix(y_test, y_predKNN)
matrice_confKNN=pd.DataFrame(cmKNN,index=['1','2','3'],columns=['1','2','3'])
matrice_confKNN

,1,2,3
1,56,375,3329
2,187,2770,36533
3,405,8687,238150


In [15]:
#confusion matrix
cmKNN = confusion_matrix(y_test, y_predKNN)
matrice_confKNN=pd.DataFrame(cmKNN,index=['1','2','3'],columns=['1','2','3'])
matrice_confKNN
#report
print(classification_report(y_test,y_predKNN))

              precision    recall  f1-score   support

           1       0.09      0.01      0.03      3760
           2       0.23      0.07      0.11     39490
           3       0.86      0.96      0.91    247242

    accuracy                           0.83    290492
   macro avg       0.39      0.35      0.35    290492
weighted avg       0.76      0.83      0.79    290492



In [ ]:
#NB
from sklearn.naive_bayes import GaussianNB
classifierGNB = GaussianNB()
classifierGNB.fit(X_train, y_train)
y_predGNB = classifierGNB.predict(X_test)
#accuracy
accGNB = accuracy_score(y_test, y_predGNB)
print("accuracy_score =",accGNB)
#confusion matrix
cmGNB = confusion_matrix(y_test, y_predGNB)
matrice_confGNB=pd.DataFrame(cmGNB,index=['1','2','3'],columns=['1','2','3'])
matrice_confGNB
#report
print(classification_report(y_test,y_predGNB))